# Fixed Coupon Bond

The goal of the project is to compute the market value of a portfolio if fixed coupon bonds taking into account the issuer credit risk. The input parameters to derive this curve are:

* The par rate of a set of Overnight Index Swaps (i.e. their market quotation)
* A set of survival probabilities and the recovery rate of the issuer
* The static data of the pool of bonds (nominal, start date, end date, coupons, payment frequency)

### Notes and hints

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for evaluation of risky flows.

All maturities are expressed in months and rates are expressed as fractions of one (i.e. 0.01 means 1%).

Remember to reuse the code we developed during the lessons in `finmarkets.py` as much as possible, instead of trying to rewrite everything from scratch!

In [1]:
from datetime import date
from dateutil.relativedelta import relativedelta

today = date(2013, 10, 31)

ois_quotes = [
    {'maturity': 1, 'rate': 0.00106},
    {'maturity': 2, 'rate': 0.00114},
    {'maturity': 3, 'rate': 0.00115},
    {'maturity': 4, 'rate': 0.00117},
    {'maturity': 5, 'rate': 0.00119},
    {'maturity': 6, 'rate': 0.00121},
    {'maturity': 7, 'rate': 0.00122},
    {'maturity': 8, 'rate': 0.00124},
    {'maturity': 9, 'rate': 0.00128},
    {'maturity': 10, 'rate': 0.00131},
    {'maturity': 11, 'rate': 0.00135},
    {'maturity': 12, 'rate': 0.00138},
    {'maturity': 15, 'rate': 0.00152},
    {'maturity': 18, 'rate': 0.00166},
    {'maturity': 21, 'rate': 0.00184},
    {'maturity': 24, 'rate': 0.00206},
    {'maturity': 36, 'rate': 0.00344},
    {'maturity': 48, 'rate': 0.00543},
    {'maturity': 60, 'rate': 0.00756},
    {'maturity': 72, 'rate': 0.00967},
    {'maturity': 84, 'rate': 0.01162},
    {'maturity': 96, 'rate': 0.0134},
    {'maturity': 108, 'rate': 0.01502},
    {'maturity': 120, 'rate': 0.01649},
    {'maturity': 132, 'rate': 0.01776},
    {'maturity': 144, 'rate': 0.01888},
    {'maturity': 180, 'rate': 0.02137},
    {'maturity': 240, 'rate': 0.02322},
    {'maturity': 300, 'rate': 0.02389},
    {'maturity': 360, 'rate': 0.02416},
]

survival_probabilities = [
    {'date': date(2014, 12, 20), 'ndp': 0.972159727015014},
    {'date': date(2015, 12, 20), 'ndp': 0.942926329174406},
    {'date': date(2016, 12, 20), 'ndp': 0.913448056250137},
    {'date': date(2018, 12, 20), 'ndp': 0.855640452819766},
    {'date': date(2023, 12, 20), 'ndp': 0.732687779675469},
    {'date': date(2033, 12, 20), 'ndp': 0.539046016487758},
]

bonds_to_price = [
    {'nominal': 4972284.02, 'start_date': date(2010, 3, 1), 'end_date': date(2021, 9, 1), 'coupon_frequency': 3, 'coupon': 0.035, 'recovery': 0.2},
    {'nominal': 7344328.27, 'start_date': date(2009, 7, 1), 'end_date': date(2022, 7, 1), 'coupon_frequency': 3, 'coupon': 0.035, 'recovery': 0.2},
    {'nominal': 7172290.19, 'start_date': date(2013, 1, 1), 'end_date': date(2017, 7, 1), 'coupon_frequency': 6, 'coupon': 0.02, 'recovery': 0.4},
    {'nominal': 7065224.23, 'start_date': date(2010, 3, 1), 'end_date': date(2014, 9, 1), 'coupon_frequency': 6, 'coupon': 0.02, 'recovery': 0.4},
    {'nominal': 5256452.14, 'start_date': date(2011, 7, 1), 'end_date': date(2016, 7, 1), 'coupon_frequency': 6, 'coupon': 0.02, 'recovery': 0.4},
    {'nominal': 2689680.89, 'start_date': date(2009, 9, 1), 'end_date': date(2024, 9, 1), 'coupon_frequency': 6, 'coupon': 0.02, 'recovery': 0.6},
    {'nominal': 3593518.71, 'start_date': date(2010, 7, 1), 'end_date': date(2019, 7, 1), 'coupon_frequency': 12, 'coupon': 0.02, 'recovery': 0.6},
    {'nominal': 6993589.53, 'start_date': date(2011, 1, 1), 'end_date': date(2019, 1, 1), 'coupon_frequency': 12, 'coupon': 0.02, 'recovery': 0.6},
    {'nominal': 6684377.52, 'start_date': date(2009, 9, 1), 'end_date': date(2021, 9, 1), 'coupon_frequency': 12, 'coupon': 0.02, 'recovery': 0.6},
    {'nominal': 6896199.04, 'start_date': date(2010, 7, 1), 'end_date': date(2018, 7, 1), 'coupon_frequency': 12, 'coupon': 0.027, 'recovery': 0.4},
    {'nominal': 2587984.6, 'start_date': date(2011, 10, 1), 'end_date': date(2020, 10, 1), 'coupon_frequency': 12, 'coupon': 0.02, 'recovery': 0.4},
    {'nominal': 3621656.1, 'start_date': date(2012, 6, 1), 'end_date': date(2016, 6, 1), 'coupon_frequency': 6, 'coupon': 0.027, 'recovery': 0.4},
    {'nominal': 3146567.47, 'start_date': date(2011, 6, 1), 'end_date': date(2022, 6, 1), 'coupon_frequency': 6, 'coupon': 0.018, 'recovery': 0.2},
    {'nominal': 6452721.61, 'start_date': date(2009, 4, 1), 'end_date': date(2019, 4, 1), 'coupon_frequency': 3, 'coupon': 0.018, 'recovery': 0.2},
    {'nominal': 3418346.24, 'start_date': date(2010, 5, 1), 'end_date': date(2016, 5, 1), 'coupon_frequency': 3, 'coupon': 0.018, 'recovery': 0.2},
]

First of all we need to determine the discount curve from the market quotes of a set of OIS using the bootstrap technique. Also, before defining the discount curve I have checked the minimization results).

In [2]:
from finmarkets import DiscountCurve, OvernightIndexSwap, generate_swap_dates

pillar_dates = [today]

swaps = []

for quote in ois_quotes:
    swap = OvernightIndexSwap(
        1e6, 
        generate_swap_dates(today, quote['maturity']),
        quote['rate'])
    
    swaps.append(swap)
    pillar_dates.append(swap.payment_dates[-1])

pillar_dates = sorted(pillar_dates)

In [3]:
def objective_function(x):
    
    curve = DiscountCurve(today, pillar_dates, x)
    
    sum_sq = 0.0
    
    for swap in swaps:
        sum_sq += swap.npv(curve) ** 2
        
    return sum_sq 

In [4]:
from scipy.optimize import minimize

x0 = [1.0 for i in range(len(pillar_dates))]
bounds = [(0.01, 100.0) for i in range(len(pillar_dates))]
bounds[0] = (1.0, 1.0)

result = minimize(objective_function, x0, bounds=bounds)

In [5]:
print (result.x)
print (objective_function(x0))
print (objective_function(result.x))

from matplotlib import pyplot as plt
plt.plot(pillar_dates, result.x, marker='o')
plt.show()

[1.         0.99991167 0.99980687 0.99970619 0.99961015 0.99950111
 0.999392   0.99928207 0.99916713 0.99903027 0.998895   0.99874906
 0.99860278 0.99807334 0.99748678 0.99674625 0.99583439 0.98958294
 0.97817155 0.9622335  0.94245275 0.91997975 0.89555481 0.86968378
 0.84283835 0.81600063 0.78928833 0.7134728  0.61195787 0.53249163
 0.46686384]
1398807951100.5298
0.0006561289593822537


<Figure size 640x480 with 1 Axes>

In [6]:
discount_curve = DiscountCurve(today, pillar_dates, result.x)

Then ww need to determine the credit curve from the non-default probabilities given in input.

In [7]:
from finmarkets import CreditCurve

date=[today]

ndp=[1.0]

for quote in survival_probabilities:
    date.append(quote['date'])
    ndp.append(quote['ndp'])
    
cc = CreditCurve(date, ndp)

In the end a new class to price the bonds has been developed. The same class is used later to compute the portfolio value.

In [17]:
from math import exp

class Price:
    
    def __init__(self, notional, d1, d2, today, 
                 coupon, frequency, recovery):

        self.notional = notional
        self.d1 = d1
        self.d2 = d2
        self.today = today
        self.coupon = coupon
        self.frequency = frequency
        self.recovery = recovery
           
    def cpd(self):
        coupon_dates = []
        now = self.d1
        
        while now + relativedelta(months=self.frequency) < self.d2:
            now += relativedelta(months=self.frequency)
            coupon_dates.append(now)
            
        coupon_dates.append(self.d2)
        coupon_dates = [i for i in coupon_dates if i >= self.today]
        
        return coupon_dates
    
    def haz(self, end):
        hazard=0.0
        t=0
        
        while t <= end: 
            hazard += cc.ndp(today+relativedelta(days=t)) - \
                      cc.ndp(today+relativedelta(days=t+1))
            t += 1
        return hazard
        
    def pricing(self):            
        price = 0
        for d in self.cpd():
            price += self.coupon * self.notional * discount_curve.df(d) * cc.ndp(d)
        
        price += self.notional * discount_curve.df(d) * cc.ndp(d)
        
        recovery = 0 
        now = today
        while now + relativedelta(days=1) <= self.d2:
            recovery += self.recovery * self.notional * discount_curve.df(now) * (cc.ndp(d) - cc.ndp(d+relativedelta(days=1)))
            now += relativedelta(days=1)
        price += recovery
        return price

In [19]:
#payment_dates=[]

#payments_values=[]
ptf_price=0.0

for quote in bonds_to_price:
   
    ptf = Price(
        quote['nominal'],
        quote['start_date'],
        quote['end_date'],
        today,        
        quote['coupon'], 
        quote['coupon_frequency'], 
        quote['recovery']        
    )

    ptf_price += ptf.pricing()

print ("The market value of the portfolio is {:.2f}".format(ptf_price))

The market value of the portfolio is 89995036.71
